In [2]:
import numpy as np
from astropy.io import fits
from astropy.table import Table, Column
from astropy.wcs import WCS

In [3]:
#dir = 'fitsdata/'
#file = 'NGC4047.co.smo7_dil.mom0.fits.gz'
dir = '../img_cocube/fitsdata/'
file = 'NGC4047.co.smo7msk.K.fits.gz'
hdu=fits.open(dir+file)[0]
data=hdu.data
bunit = hdu.header['BUNIT']
w=WCS(hdu)
data.shape
ndim=len(data.shape)
iscube = (ndim > 2 and data.shape[ndim-3] > 1)
if iscube:
    print('This is a data cube')
    data=np.squeeze(data)
    ix,iy,iz=np.meshgrid(np.arange(data.shape[2]),np.arange(data.shape[1]),
                         np.arange(data.shape[0]),indexing='ij')
    tab=Table([np.ravel(ix),np.ravel(iy),np.ravel(iz)],
              names=('ix','iy','iz'),dtype=('i4','i4','i4'))
    if ndim == 3:
        wcsin = (np.array([tab['ix'],tab['iy'],tab['iz']])).T
    if ndim == 4:
        wcsin = (np.array([tab['ix'],tab['iy'],tab['iz'],0*tab['ix']])).T
else:
    print('This is an image')
    data=np.squeeze(data)
    ix,iy=np.meshgrid(np.arange(data.shape[1]),np.arange(data.shape[0]))
    tab=Table([np.ravel(ix),np.ravel(iy)],names=('ix','iy'),dtype=('i4','i4'))
    if ndim == 2:
        wcsin = (np.array([tab['ix'],tab['iy']])).T
        refin = []
    if ndim == 3:
        wcsin = (np.array([tab['ix'],tab['iy'],0*tab['ix']])).T
    if ndim == 4:
        wcsin = (np.array([tab['ix'],tab['iy'],0*tab['ix'],0*tab['ix']])).T
wcsout = w.wcs_pix2world(wcsin,0)
col_ra = Column(wcsout.T[0]-w.wcs.crval[0], name='ra_off', dtype='f4', unit='deg')
col_dec = Column(wcsout.T[1]-w.wcs.crval[1], name='dec_off', dtype='f4', unit='deg')
tab.add_columns([col_ra,col_dec])
if iscube:
    col_vel = Column(wcsout.T[2]/1000., name='vel', dtype='f4', unit='km/s')
    tab.add_column(col_vel)
# Use order = 'F' because the velocity axis is first
col_data = Column(np.ravel(data,order='F'), name='imgdata', dtype='f4', unit=bunit)
tab.add_column(col_data)
print(tab[20:50])
print('RA ref is',w.wcs.crval[0])
print('DEC ref is',w.wcs.crval[1])
tab.write('test.hdf5', path='data', overwrite=True, serialize_meta=True, compression=True)


This is a data cube
 ix  iy  iz   ra_off   dec_off    vel   imgdata
               deg       deg     km / s    K   
--- --- --- --------- ---------- ------ -------
  0   0  20 0.0336125 -0.0222271 3380.0     nan
  0   0  21 0.0336125 -0.0222271 3400.0     nan
  0   0  22 0.0336125 -0.0222271 3420.0     nan
  0   0  23 0.0336125 -0.0222271 3440.0     nan
  0   0  24 0.0336125 -0.0222271 3460.0     nan
  0   0  25 0.0336125 -0.0222271 3480.0     nan
  0   0  26 0.0336125 -0.0222271 3500.0     nan
  0   0  27 0.0336125 -0.0222271 3520.0     nan
  0   0  28 0.0336125 -0.0222271 3540.0     nan
  0   0  29 0.0336125 -0.0222271 3560.0     nan
... ... ...       ...        ...    ...     ...
  0   0  40 0.0336125 -0.0222271 3780.0     nan
  0   0  41 0.0336125 -0.0222271 3800.0     nan
  0   0  42 0.0336125 -0.0222271 3820.0     nan
  0   0  43 0.0336125 -0.0222271 3840.0     nan
  0   1   0 0.0336127 -0.0219493 2980.0     nan
  0   1   1 0.0336127 -0.0219493 3000.0     nan
  0   1   2 0.033612

In [ ]:
def fits2hdf(stride=[2,2,1],keepref=True,keepnan=True):
    hdu = fits.open(filename)[0]
    data=hdu.data
    ndim=len(data.shape)
    if ndim > 2 and data.shape[ndim-3] > 1:
        if ndim > 3:
            data=np.squeeze(data)
            if len(data.shape) > 3:
                raise ('Data cannot be squeezed to three dimensions')